# Case study: maintenance schedule

This case study illustrates the use of the staircase package to create the union of intervals, to remove overlaps between them.  In this example we have an asset composed of several components, named A, B, C, D and E. Each of these components can undergo multiple periods of maintenance, as specified in a maintenance schedule  If any component is under maintenance then the entire asset is considered under maintenance.  The problem is essentially to create the union of the intervals specified in the maintenance schedule

In [ ]:
import pandas as pd
import staircase as sc
import matplotlib.pyplot as plt

We begin by importing the maintenance schedule data into a pandas.DataFrame instance.  Each row corresponds to a period of maintenance for a component.  The first column identifies the component and the second and third columns indicate the start and end points of each maintenance period.

In [ ]:
data = pd.read_csv("./data/maintenance_schedule.csv", parse_dates=['start', 'end'], dayfirst=True)
data

We'll start by creating a Stairs object to model the number of components under maintenance at any point in time as a step function.

In [ ]:
components_under_maintenance = sc.Stairs(use_dates=True).layer(data.start, data.end)
components_under_maintenance.plot()

The default plot size is a bit too small so let's create a bigger axes and pass it to the plot method.

In [ ]:
fig, ax = plt.subplots(figsize=(20,3))
components_under_maintenance.plot(ax)

Our asset will be under maintenance whenever this step function is non-zero.  There are several ways this can be achieved.  Each option below returns a staircase.Stairs instance:

- asset_under_maintenance = components_under_maintenance > 0
- asset_under_maintenance = components_under_maintenance != 0
- asset_under_maintenance = ~(components_under_maintenance == 0)
- asset_under_maintenance = components_under_maintenance >= 1
- asset_under_maintenance = components_under_maintenance.make_boolean()

Let's use the first option and plot it together with the first step function:

In [ ]:
asset_under_maintenance = components_under_maintenance > 0

fig, ax = plt.subplots(figsize=(20,3))
components_under_maintenance.plot(ax)
asset_under_maintenance.plot(ax)

Next we want to extract this interval data from our new step function.  One way is to use the step_changes() method to get the points at which the function changes value:

In [ ]:
asset_under_maintenance.step_changes()

However a better way to extract the data would be to use the to_dataframe method:

In [ ]:
asset_under_maintenance.to_dataframe()

Lastly we can filter the rows to only include those parts of the function which are equal to one, and drop the value column.  We can then write the dataframe to csv if necessary.

In [ ]:
asset_under_maintenance.to_dataframe().query('value == 1').drop(columns='value')